# Linear Algebra Group Project
#### Group Members: Corey Dobbs and Emma Rasmussen 

### Problem 1
#### Part a: Write as a variational problem 
 
We are given the following for our boundary problem: 
$$ 
-u''(x) + V(x)u'(x) = f(x), x\epsilon [0,1]\\
u(0) = u(1) = 0\\
V(x) = \gamma, f(x) = 1\\
$$ 
We first start by picking a test function, $\phi(x)$. We then mupliply our initial function by this value and interate over x to get the following: 
$$ 
\quad -\int_{0}^{1} u''(x)\phi(x) \, dx + \int_{0}^{1} u'(x)V(x)\phi(x) \, dx = \int_{0}^{1} f(x)\phi(x) \, dx
$$
Focusing on the first term, we can use integration by parts to expand that to: 
$$
\left. -\phi \right|_{0}^{1} + \int_{0}^{1} u'(x)\phi'(x) \,dx 
$$
The first term of the above equation will go to zero, so getting rid of that and plugging our new expression back into our original intergral equation, we would end up with: 
$$
\quad \int_{0}^{1} u'(x)\phi'(x) \,dx + \int_{0}^{1} u'(x)V(x)\phi(x) \, dx = \int_{0}^{1} f(x)\phi(x) \, dx
$$ 
Where the terms, respecitvely, correspond to: 
$$ 
A_1[\phi(x),u(x)] + A_2[\phi(x),u(x)] = F[\phi(x)] 
$$ 

#### Part b: Expand u as a linear combination

Next, we will be expanding u as the given linear combination, $u = \sum_{i=1}^{m} u_i\phi_i(x)$. We will stop writing the x dependence of $\phi$ for simplicity, but note that it still exists. Also, we will assume that we are choosing $\phi$ from a finite dimensional basis $\{\phi_1, \phi_2, ...,\phi_m\}$:
$$
A_1[\phi_j,\sum_{i=1}^{m} u_i\phi_i] + A_2[\phi_j,\sum_{i=1}^{m} u_i\phi_i] = F[\phi_j] 
$$
Since $u_i$ is a scalar, we can pull that and the sum out and simplify to: 
$$ 
\sum_{i=1}^{m} u_i(A_1[\phi_j,\phi_i] + A_2[\phi_j,\phi_i]) = F[\phi_j] 
$$ 
In this expression, $u_i$ corresponds to $x$, $A_1[\phi_j,\phi_i] + A_2[\phi_j,\phi_i]$ corersponds to $A_{ij}$, and $F[\phi_j] $ corresponds to $b$, which shows that our problem is in the form $Ax-b$. 

#### Part c: Implement a driver routine that will return A and b given inputs n and gamma.  The matrix A should be implemented as a sparse representation in your environment.

In [3]:
import numpy as np
from scipy.sparse import csr_matrix

def driver(n, g):
    A = csr_matrix((n,n))
    

In [3]:
from scipy.sparse import csr_matrix, diags
import numpy as np

def driver( n, gamma ):
    # A1 = csr_matrix((n+1,n+1))
    # A2 = csr_matrix((n+1,n+1))
    A1 = np.zeros( (n+1,n+1) )
    A2 = np.zeros( (n+1,n+1) )
    for i in range( 1, n ):
        phi_hati = np.zeros( n )
        phi_hati[i] = 0.5

        phi_hatpi = np.zeros( n )
        phi_hatpi[i-1] = n
        phi_hatpi[i] = -n
        
        for j in range( 1, n ):
            phi_hatpj = np.zeros( n )
            phi_hatpj[j-1] = n
            phi_hatpj[j] = -n

            A1[i,j] = np.sum( phi_hatpi * phi_hatpj ) / n
            A2[i,j] = gamma * np.sum( phi_hati * phi_hatpj ) / n

            
            

    A = A1 + A2
    b = np.ones( n )
    return A, b, A1, A2

A, b, A1, A2 = driver( 3, gamma=1 )
# print(A1)
# print(A2)
print(A1+A2)


[[ 0.   0.   0.   0. ]
 [ 0.   5.5 -2.5  0. ]
 [ 0.  -3.   5.5  0. ]
 [ 0.   0.   0.   0. ]]


In [22]:
def mygmres( I, b, x0, n, A ):

    r0 = b #- A @ x0
    beta = np.linalg.norm( r0, 2 )
    
    v1 = (r0 / beta).T
    print(f"v1={v1}")
    v = np.zeros( (len(v1),n) )
    v[:,0] = v1

    H = csr_matrix( (n+1,n) )

    for j in range( n ):
        wj = A @ v[j,:]
        for i in range( j ):
            H[i,j] = np.dot( wj, v[j,:] )
            print(H[i,j])
            wj = H[i,j] * v[j,:]

        H[j+1,j] = np.linalg.norm( wj, 2 )
        print(H[j+1,j])
        if H[j+1,j]==0:
            n = j
            break

        v[j+1] = wj / H[j+1,j]
    return v, H

A = np.array([[1,4,7],[2,9,7],[5,8,3]])
b = np.array([1,8,2]).T
x0=0

vtest, Htest = mygmres( 3, b, x0, 3, A )

v1=[0.12038585 0.96308682 0.24077171]
0.659380473395787
11.133333333333333
11.133333333333331
11.133333333333336
11.133333333333338
11.13333333333334


/home/byu.local/cedobbs/.local/lib/python3.8/site-packages/scipy/sparse/_index.py:103: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


IndexError: index 3 is out of bounds for axis 0 with size 3

In [13]:
print(vtest)
print(Htest)

[[0.12038585 0.96308682 0.24077171]
 [0.37656678 0.70506121 0.60090444]
 [0.37656678 0.70506121 0.60090444]
 [0.37656678 0.70506121 0.60090444]]
  (0, 1)	16.362562588265504
  (0, 2)	16.362562588265497
  (1, 0)	15.02558205002485
  (1, 2)	16.362562588265494
  (2, 1)	16.362562588265508
  (3, 2)	16.36256258826549
